# Final Capstone Project

In this notebook we will cluster neighbourhood according to their trending venues and figure out in which neighbourhood should a restaurtant open

In [10]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!conda install folium -c conda-forge
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import json


Solving environment: done

# All requested packages already installed.



In [11]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    

Data downloaded!


In [12]:
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    

In [13]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
CLIENT_ID = 'RB13D141CRD5NO4VQVKV2UILVQPVZUBCY5SXRKAXEZ1FR2ZJ' 
CLIENT_SECRET = 'TP5TZHBR0NX3P44YC2NICKCHWHROVRVS431RSAYFYE4KLAWU' 
VERSION = '20180605'
LIMIT=20

In [15]:
def getTrendingVenues(names, latitudes, longitudes, radius=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(results)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
NY_venues = getTrendingVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
print(NY_venues.shape)
NY_venues.head()

Wakefield
[]
Co-op City
[]
Eastchester
[]
Fieldston
[]
Riverdale
[]
Kingsbridge
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '56e89785cd10a6e6398f0628', 'name': 'Dollar Tree', 'location': {'address': '5687 Broadway', 'lat': 40.88171500958054, 'lng': -73.9031865652043, 'labeledLatLngs': [{'label': 'display', 'lat': 40.88171500958054, 'lng': -73.9031865652043}], 'distance': 31, 'postalCode': '10463', 'cc': 'US', 'city': 'Bronx', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['5687 Broadway', 'Bronx, NY 10463', 'United States']}, 'categories': [{'id': '52dea92d3cf9994f4e043dbb', 'name': 'Discount Store', 'pluralName': 'Discount Stores', 'shortName': 'Discount Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/discountstore_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-56e89785cd10a6e6398f0628-0

[]
University Heights
[]
Morris Heights
[]
Fordham
[]
East Tremont
[]
West Farms
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bd1aabea8b3a5932617665f', 'name': "Jimbo's Hamburger Palace", 'location': {'address': '346 Devoe Ave', 'lat': 40.839217439995124, 'lng': -73.87803216408489, 'labeledLatLngs': [{'label': 'display', 'lat': 40.839217439995124, 'lng': -73.87803216408489}], 'distance': 37, 'postalCode': '10460', 'cc': 'US', 'city': 'Bronx', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['346 Devoe Ave', 'Bronx, NY 10460', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d147941735', 'name': 'Diner', 'pluralName': 'Diners', 'shortName': 'Diner', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_', 'suffix': '.png'}, 'primary': True}], 'delivery': {'id': '518546', 'url': 'https://www.seamless.com/menu/jimbos-hamburger-palace-346-devoe-a

[]
Bay Ridge
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '57bf2b24498e2a2800681ed3', 'name': 'Georgian Dream Cafe and Bakery', 'location': {'address': '8309 3rd Ave', 'lat': 40.625586, 'lng': -74.030196, 'labeledLatLngs': [{'label': 'display', 'lat': 40.625586, 'lng': -74.030196}], 'distance': 43, 'postalCode': '11209', 'cc': 'US', 'city': 'Brooklyn', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['8309 3rd Ave', 'Brooklyn, NY 11209', 'United States']}, 'categories': [{'id': '5293a7d53cf9994f4e043a45', 'name': 'Caucasian Restaurant', 'pluralName': 'Caucasian Restaurants', 'shortName': 'Caucasian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caucasian_', 'suffix': '.png'}, 'primary': True}], 'delivery': {'id': '1024692', 'url': 'https://www.seamless.com/menu/georgian-cuisine-8309-3rd-ave-brooklyn/1024692?affiliate=1131&utm_source=foursquare-affi

[]
Manhattan Terrace
[]
Flatbush
[]
Crown Heights
[]
East Flatbush
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4c28daf33492a5934d02b828', 'name': 'Key Food', 'location': {'address': '4209 Avenue D', 'crossStreet': 'btw E 42nd St & E 43rd St', 'lat': 40.64180595416915, 'lng': -73.93649015186261, 'labeledLatLngs': [{'label': 'display', 'lat': 40.64180595416915, 'lng': -73.93649015186261}], 'distance': 34, 'postalCode': '11203', 'cc': 'US', 'city': 'Brooklyn', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['4209 Avenue D (btw E 42nd St & E 43rd St)', 'Brooklyn, NY 11203', 'United States']}, 'categories': [{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'Supermarket', 'pluralName': 'Supermarkets', 'shortName': 'Supermarket', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups

[]
Cobble Hill
[]
Carroll Gardens
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4a2aab17f964a52033961fe3', 'name': 'Carroll Park', 'location': {'address': '290 President St', 'crossStreet': 'btwn Smith & Court', 'lat': 40.68090072479618, 'lng': -73.99498436689395, 'labeledLatLngs': [{'label': 'display', 'lat': 40.68090072479618, 'lng': -73.99498436689395}], 'distance': 48, 'postalCode': '11231', 'cc': 'US', 'city': 'Brooklyn', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['290 President St (btwn Smith & Court)', 'Brooklyn, NY 11231', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4a2aab17f964a52033961fe3-0'},

...

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '51f161e2498ea191a1ad1b44', 'name': "Dag's Patio Cafe", 'location': {'address': '342 E 47th St', 'crossStreet': '1st Avenue', 'lat': 40.752200469485025, 'lng': -73.96819788811214, 'labeledLatLngs': [{'label': 'display', 'lat': 40.752200469485025, 'lng': -73.96819788811214}], 'distance': 44, 'postalCode': '10017', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['342 E 47th St (1st Avenue)', 'New York, NY 10017', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d16d941735', 'name': 'Café', 'pluralName': 'Cafés', 'shortName': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}, 'venuePage': {'id': '77374025'}}, 'referralId': 'e-0-51f161e2498ea191a1ad1b44-0'}]
Tudor City
[]
Stuyvesant T

[]
Weeksville
[]
Broadway Junction
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '57e14af9498e3d6769fe65fd', 'name': 'MTA Bus - B20/B25/Q24 @ Fulton St & Van Sinderen Ave', 'location': {'address': 'Fulton Street', 'crossStreet': 'Fulton Street and Van Sinderen Avenue', 'lat': 40.67787540586492, 'lng': -73.90382251519794, 'labeledLatLngs': [{'label': 'display', 'lat': 40.67787540586492, 'lng': -73.90382251519794}], 'distance': 42, 'postalCode': '11233', 'cc': 'US', 'city': 'Brooklyn', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['Fulton Street (Fulton Street and Van Sinderen Avenue)', 'Brooklyn, NY 11233', 'United States']}, 'categories': [{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'Bus Stop', 'pluralName': 'Bus Stops', 'shortName': 'Bus Stop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/busstation_', 'suffix': '.png'}, 'primary': True}], 'pho

[]
Egbertville
[]
Roxbury
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '50011a44e4b0a74a522da159', 'name': "Pebble's", 'location': {'lat': 40.56713974144648, 'lng': -73.89205079565416, 'labeledLatLngs': [{'label': 'display', 'lat': 40.56713974144648, 'lng': -73.89205079565416}], 'distance': 27, 'postalCode': '11697', 'cc': 'US', 'city': 'Breezy Point', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['Breezy Point, NY 11697', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d116941735', 'name': 'Bar', 'pluralName': 'Bars', 'shortName': 'Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-50011a44e4b0a74a522da159-0'}]
Homecrest
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globa

[]
Highland Park
[]
Madison
[]
Bronxdale
[]
Allerton
[]
Kingsbridge Heights
[]
Erasmus
[]
Hudson Yards
[]
Hammels
[]
Bayswater
[]
Queensbridge
[]
Fox Hills
[]
(299, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kingsbridge,40.881687,-73.902818,Dollar Tree,40.881715,-73.903187,Discount Store
1,Kingsbridge,40.881687,-73.902818,Mattress Firm,40.881580,-73.903277,Mattress Store
2,Woodlawn,40.898273,-73.867315,Rambling House,40.898439,-73.867197,Pub
3,Woodlawn,40.898273,-73.867315,Behan's Pub,40.898585,-73.867507,Bar
4,Woodlawn,40.898273,-73.867315,Just One Tavern,40.897872,-73.867437,Beer Bar


In [18]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bagel Shop,Bakery,...,Thrift / Vintage Store,Toy / Game Store,Train Station,Varenyky restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Kingsbridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Kingsbridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Woodlawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Woodlawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Woodlawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
neighborhoods.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bay Ridge,Pool Hall,Caucasian Restaurant,Yoga Studio,Donut Shop,Food Court
1,Bay Terrace,Bus Stop,Convenience Store,Food Truck,Food Court,Food
2,Bedford Stuyvesant,Gym / Fitness Center,Yoga Studio,Dry Cleaner,Food Truck,Food Court
3,Brighton Beach,Restaurant,Liquor Store,Varenyky restaurant,Yoga Studio,Food
4,Broadway Junction,Bus Stop,Convenience Store,Food Truck,Food Court,Food


In [22]:
# set number of clusters
kclusters = 10

NY_grouped_clustering = NY_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([9, 5, 3, 9, 5, 9, 9, 4, 9, 9], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()
NY_merged = neighborhoods


NY_merged = NY_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
NY_merged.dropna(inplace=True)

print(NY_merged)
 # check the last columns!



           Borough        Neighborhood   Latitude  Longitude  Cluster Labels  \
5            Bronx         Kingsbridge  40.881687 -73.902818             2.0   
7            Bronx            Woodlawn  40.898273 -73.867315             9.0   
12           Bronx         City Island  40.847247 -73.786488             1.0   
18           Bronx          West Farms  40.839475 -73.877745             1.0   
21           Bronx          Mott Haven  40.806239 -73.916100             2.0   
26           Bronx           Soundview  40.821012 -73.865746             9.0   
31           Bronx  Westchester Square  40.840619 -73.842194             9.0   
35           Bronx      Spuyten Duyvil  40.881395 -73.917190             9.0   
41           Bronx           Olinville  40.871371 -73.863324             9.0   
44           Bronx           Unionport  40.829774 -73.850535             4.0   
46        Brooklyn           Bay Ridge  40.625801 -74.030621             9.0   
48        Brooklyn         Sunset Park  

In [37]:
# create map
latitude =40.7128
longitude = -74

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighborhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 0, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
173,Steinway,Sushi Restaurant,Greek Restaurant,Rock Climbing Spot,Yoga Studio,Donut Shop
271,Sutton Place,Sushi Restaurant,Yoga Studio,French Restaurant,Food Court,Food


In [28]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 1, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,City Island,Bar,Diner,Grocery Store,Yoga Studio,Food Truck
18,West Farms,Diner,Yoga Studio,Dry Cleaner,Food Truck,Food Court
108,Yorkville,Diner,Gym,Bank,Yoga Studio,Dry Cleaner
147,Ozone Park,Grocery Store,Yoga Studio,Dry Cleaner,Food Truck,Food Court
288,Roxbury,Bar,Yoga Studio,Fried Chicken Joint,Food Truck,Food Court


In [29]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 2, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Kingsbridge,Mattress Store,Discount Store,Dry Cleaner,Food Truck,Food Court
21,Mott Haven,Discount Store,Yoga Studio,Dry Cleaner,Food Truck,Food Court
199,Stapleton,Coffee Shop,Department Store,Discount Store,Yoga Studio,Falafel Restaurant


In [30]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 3, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
63,Bedford Stuyvesant,Gym / Fitness Center,Yoga Studio,Dry Cleaner,Food Truck,Food Court
273,Turtle Bay,Café,Yoga Studio,Dry Cleaner,Food Truck,Food Court
276,Flatiron,Women's Store,Gym / Fitness Center,Shoe Store,Sporting Goods Shop,Café


In [31]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 4, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,Unionport,Pizza Place,Yoga Studio,Donut Shop,Food Court,Food
106,East Harlem,Pet Store,Pizza Place,Donut Shop,Food Court,Food
118,East Village,Hookah Bar,Pizza Place,Filipino Restaurant,Bagel Shop,Donut Shop
247,Carnegie Hill,Pizza Place,Italian Restaurant,Yoga Studio,Donut Shop,Food Court
291,Prince's Bay,Pizza Place,Bagel Shop,Yoga Studio,Donut Shop,Food Court


In [32]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 5, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
77,Manhattan Beach,Ice Cream Shop,Bus Stop,Deli / Bodega,Department Store,Dessert Shop
161,Oakland Gardens,Bus Stop,Convenience Store,Food Truck,Food Court,Food
175,Bay Terrace,Bus Stop,Convenience Store,Food Truck,Food Court,Food
205,Port Richmond,Bus Stop,Playground,Donut Shop,Food Court,Food
235,Bay Terrace,Bus Stop,Convenience Store,Food Truck,Food Court,Food
282,Broadway Junction,Bus Stop,Convenience Store,Food Truck,Food Court,Food


In [33]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 6, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
123,West Village,Chinese Restaurant,Bakery,Cocktail Bar,Dry Cleaner,Food Truck
133,Howard Beach,Chinese Restaurant,Yoga Studio,Dry Cleaner,Food Truck,Food Court
186,Ravenswood,Chinese Restaurant,Yoga Studio,Dry Cleaner,Food Truck,Food Court


In [34]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 7, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
155,Glen Oaks,Park,Yoga Studio,Donut Shop,Food Court,Food
195,North Corona,Park,Baseball Field,Yoga Studio,Dry Cleaner,Food Court
268,Concourse Village,Park,Yoga Studio,Donut Shop,Food Court,Food


In [35]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 8, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
59,Prospect Heights,Garden,Yoga Studio,Fried Chicken Joint,Food Truck,Food Court


In [36]:
#cluster1
NY_merged.loc[NY_merged['Cluster Labels'] == 9, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Woodlawn,Bar,Convenience Store,Beer Bar,Pub,Yoga Studio
26,Soundview,Convenience Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
31,Westchester Square,Sandwich Place,Yoga Studio,Concert Hall,Food Court,Food
35,Spuyten Duyvil,Bus Line,Yoga Studio,Convenience Store,Food Truck,Food Court
41,Olinville,Bakery,Food,Fried Chicken Joint,Food Truck,Food Court
46,Bay Ridge,Pool Hall,Caucasian Restaurant,Yoga Studio,Donut Shop,Food Court
48,Sunset Park,Bank,Pharmacy,Latin American Restaurant,Yoga Studio,Food Court
49,Greenpoint,Fruit & Vegetable Store,Vegetarian / Vegan Restaurant,Yoga Studio,Donut Shop,Food Court
51,Brighton Beach,Restaurant,Liquor Store,Varenyky restaurant,Yoga Studio,Food
56,East Flatbush,Supermarket,Moving Target,Yoga Studio,Dry Cleaner,Food Court
